In [1]:
#imports 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import os
import time
import wget

In [2]:
#specify the path to chromedriver.exe
driver = webdriver.Chrome('C:/Users/paula/chromedriver.exe')

#open the webpage
driver.get("https://app.powerbi.com/view?r=eyJrIjoiMTVkYjVjMmEtOWI1Mi00N2Q0LWI1YzUtMzUyMmE3NGVlMTBjIiwidCI6IjRiMWJkNWRiLTY3ODItNDY2YS1hMWM1LTRlOTc1NjQ4ZjhlNSIsImMiOjl9")
# waiting for 5 seconds for the full page to load
time.sleep(5)
#clicking the first button 
tour=driver.find_element_by_xpath("(//div[@class='visual visual-actionButton allow-deferred-rendering'])[1]").click()

#then the second button and selecting 2020
resultCountry=driver.find_element_by_xpath("(//div[@class='visual visual-actionButton allow-deferred-rendering'])[7]").click()

#SWITCH TO IFRAME
aaframe=WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it("visual-sandbox"))
yearSelect=driver.find_element_by_xpath("(//span[@class='slicerText'])[1]").click()
#SWITCHING BACK
driver.switch_to.default_content()

# clicking the dropdown button
findCountry=driver.find_element_by_xpath("(//div[@class='slicer-dropdown-menu'])[2]").click()



In [3]:
#finding Japan in the dropdown
searchbox=driver.find_element_by_xpath("(//input[@class='searchInput'])[2]")
searchbox.clear
searchbox.send_keys("japan")


In [4]:
#accesing data for japan
tick=driver.find_element_by_xpath("//div[@class='slicerCheckbox']").click()

In [5]:
graph=driver.find_element_by_xpath("(//div[@class='visual visual-areaChart allow-deferred-rendering'])[1]")


In [6]:
from selenium.webdriver import ActionChains
#Right click on the graph so I can obtain the table
actionChains = ActionChains(driver)
actionChains.context_click(graph).perform()

In [7]:
showTable=driver.find_element_by_xpath("(//div[@class='label'])[1]").click()

In [8]:
#Got the table, now I will work with BeautifulSoup as well
from bs4 import BeautifulSoup

In [9]:
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
table=soup.find_all('div',attrs={"class":"visual visual-pivotTable allow-deferred-rendering"})
headings= soup.find('div', attrs={"class":"columnHeaders"})
print(headings.text)

Selected year Previous year 2019 arrivals 


In [11]:
headingss= (soup.find('div', attrs={"class":"columnHeaders"})).find_next('div', attrs={"class": None})

In [12]:
headings_list=[i.text for i in headingss]
print(headings_list)

['Selected year\xa0', 'Previous year\xa0', '2019 arrivals\xa0']


In [13]:
allHeads=headings_list
head1=soup.find('div',attrs={"title":"shortmonth"})
print(head1.text)

shortmonth 


In [14]:
allHeads.append(head1.text)
print(allHeads)

['Selected year\xa0', 'Previous year\xa0', '2019 arrivals\xa0', 'shortmonth\xa0']


In [15]:
del (allHeads[2])
print(allHeads)

['Selected year\xa0', 'Previous year\xa0', 'shortmonth\xa0']


In [16]:
#Getting months
months_value= (soup.find('div', attrs={"class":"rowHeaders"})).find_next('div', attrs={"class": None})
shortmonth_values=[i.text for i in months_value]
print(shortmonth_values)

['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


In [17]:
#Getting numbers

a2020=driver.find_elements_by_xpath("(//div[@class='pivotTableCellWrap cell-interactive tablixAlignRight '])[position()<13]")
list2020= []
for i in a2020:
    list2020.append(str(i.text))
print(list2020)
    

['2661022', '1085147', '193658', '2917', '1663', '2565', '3782', '8658', '13684', '27386', '56673', '58673']


In [18]:
a2019=driver.find_elements_by_xpath("(//div[@class='pivotTableCellWrap cell-interactive tablixAlignRight '])[position()>12 and position()<=24]")
list2019= []
for i in a2019:
    list2019.append(str(i.text))
print(list2019)

['2689339', '2604322', '2760136', '2926685', '2773091', '2880041', '2991189', '2520134', '2272883', '2496568', '2441274', '2526387']


In [41]:
value= (list2019 + list2020)
print(value)

['2689339', '2604322', '2760136', '2926685', '2773091', '2880041', '2991189', '2520134', '2272883', '2496568', '2441274', '2526387', '2661022', '1085147', '193658', '2917', '1663', '2565', '3782', '8658', '13684', '27386', '56673', '58673']


In [42]:
from datetime import datetime
date1=['01-{0:02d}'.format(i) for i in range(1, 13)]
date2=['01-{0:02d}'.format(i) for i in range(1, 13)]
date=(date1 + date2)
print(date)

['01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07', '01-08', '01-09', '01-10', '01-11', '01-12', '01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07', '01-08', '01-09', '01-10', '01-11', '01-12']


In [47]:
year2019=[2019]*12
year2020=[2020]*12
year=(year2019 + year2020)
print(year)

[2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020]


In [49]:
#Woking with pandas to get the table nicely formatted
import pandas as pd
dff=pd.DataFrame([year,date,value]).T
heads=['year','date','value']
dff.columns=heads
display(dff)

,year,date,value
0,2019,01-01,2689339
1,2019,01-02,2604322
2,2019,01-03,2760136
3,2019,01-04,2926685
4,2019,01-05,2773091
5,2019,01-06,2880041
6,2019,01-07,2991189
7,2019,01-08,2520134
8,2019,01-09,2272883
9,2019,01-10,2496568


In [52]:
#Finally saving everything
dff.to_csv('C:/Users/paula/japan_inbound_2020.csv', encoding='utf-8')